In [30]:
#Importing libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

#Importing webpage using BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text  
soup = BeautifulSoup(source, 'lxml')
soup

#Finding the table
table1 = soup.find('table', {'class':'wikitable sortable'})

#Extracting each row
table_rows= table1.find_all('tr')
table_rows

#Create an empty list
list_postal_codes= []  # "pd.DataFrame(columns=['PostalCode', 'Borough','Neighborhood'])

#Reading attributes of each row 
for tr in table_rows:
    td = tr.find_all('td')
#Controlling empty rows
    if len(td) == 0:
        pass
    else:
#Extracting text of each attribute and processing cells without "Not assigned" Borough
        row= [i.text for i in td]
        if row[1] == 'Not assigned':
            pass
        
        else:
            
#Changing "Not assigned" in Neighborhood for the same value as Borough
            if 'Not assigned' in row[2]:
                row[2] = row[1]

#Replacing '\n' in Neighborhood for ''
            if '\n' in row[2]:
                row[2]= row[2].rstrip() #Remove '\n'
                row[2]= row[2]+', ' #Adding ', ' before applying groupby

#Adding the row to the list 'list_postal_codes'
            list_postal_codes.append(row)


#Creating Dataframe
PostalCodes = pd.DataFrame(data=list_postal_codes, columns=['PostalCode', 'Borough','Neighborhood'])
PostalCodes

#Grouping by PostalCode
df = PostalCodes.groupby(['PostalCode', 'Borough']).sum()
df= df.reset_index()

#Cleaning data. Removing ', ' from the last two characters in Neighborhood
strlen = 0
indexval = 0
for n in df.itertuples():
    indexval= n.Index
    strlen = len(n.Neighborhood)
    val = n.Neighborhood[0:strlen - 2]
    df['Neighborhood'][indexval] = val
    
    
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
#Dataframe shape
df.shape

(103, 3)

In [32]:
#Importing geocoder from given website
!wget -q -O 'geocodercsv.csv' http://cocl.us/Geospatial_data
geocodercsv = pd.read_csv('geocodercsv.csv')
geocodercsv.rename(columns={'Postal Code':'PostalCode', 'Latitude':'Latitude', 'Longitude':'Longitude'}, inplace=True)
geocodercsv.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
#Adding columns Latitute and Longitude to PostalCodes Data Frame
df['Latitude'] = ""
df['Longitude'] =  ""
df

#Setting PostalCode as Index
df.set_index('PostalCode', inplace= True)
df.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",,
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
M1E,Scarborough,"Guildwood, Morningside, West Hill",,
M1G,Scarborough,Woburn,,
M1H,Scarborough,Cedarbrae,,


In [35]:
#Assigning Latitute and Longitude to PostalCodes Data Frame
for n in geocodercsv.itertuples():
    indexval= n.PostalCode
    lon = float(n.Latitude)
    lat = float(n.Longitude)
    df['Latitude'][indexval] = lon
    df['Longitude'][indexval] = lat
#     print(n)
df= df.reset_index()
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [36]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [37]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [39]:
toronto_data = df.set_index('Borough').filter(like='Toronto', axis=0)
toronto_data= toronto_data.reset_index()
toronto_data

,Borough,PostalCode,Neighborhood,Latitude,Longitude
0,East Toronto,M4E,The Beaches,43.6764,-79.293
1,East Toronto,M4K,"The Danforth West, Riverdale",43.6796,-79.3522
2,East Toronto,M4L,"The Beaches West, India Bazaar",43.669,-79.3156
3,East Toronto,M4M,Studio District,43.6595,-79.3409
4,Central Toronto,M4N,Lawrence Park,43.728,-79.3888
5,Central Toronto,M4P,Davisville North,43.7128,-79.3902
6,Central Toronto,M4R,North Toronto West,43.7154,-79.4057
7,Central Toronto,M4S,Davisville,43.7043,-79.3888
8,Central Toronto,M4T,"Moore Park, Summerhill East",43.6896,-79.3832
9,Central Toronto,M4V,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6864,-79.4


In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [44]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: UV0DYCQGQZ4MCU2DMYXJVM2NUAZYFYN1YD3XZVM5IHH5IJPC
CLIENT_SECRET:5U1Z3IEXGO5E0BOGL2LYQE3LEI2AIOP2ICTYVOCBV33IPAIA
